In [ ]:
import petl as etl # для загрузки и обработки данных
import json # для чтения файла json 
import pandas as pd # для выгрузки таблицы в postgresql
import sqlalchemy # для создания подключения к базе данных

In [ ]:
# выгружаем данные из json файла
with open('example.json', 'r', encoding='utf-8') as f: 
    text = json.load(f)

In [90]:
type(text[0])

dict

In [ ]:
# функция для установки правильного формата дат
def make_date(item):
    day = item['bibliography']['publication']['day']
    month = item['bibliography']['publication']['month']
    year = item['bibliography']['publication']['year']
    return f"{year:04d}-{month:02d}-{day:02d}"

In [93]:
# переструктурирование json файла в словарь python
df_dict = [{
    'name': item['bibliography']['author']['name'], 
    'title': item['bibliography']['title'], 
    'publication': make_date(item),
    'downloads': item['metadata']['downloads'],
    'words': item['metrics']['statistics']['words']}
    for item in text]

In [96]:
table = etl.fromdicts(df_dict) # создание таблицы petl

In [98]:
table = etl.addfield(table, 'Last_name_letter', lambda x: x['name'][0]) # добавление столбца с первой буквой фамилии

In [ ]:
table.toxlsx('example.xlsx', mode='overwrite') # экспорт в Excel файл

In [ ]:
engine = sqlalchemy.create_engine('postgresql://{user}:{user_password}@{url}:{port}/{database_name}') # подключение к базе данных

In [ ]:
df = pd.DataFrame(columns=table[0], data=table[1:]) # создание DataFrame из petl-таблицы

In [100]:
df

,name,title,publication,downloads,words,Last_name_letter
0,"Austen, Jane",Pride and Prejudice,1998-06-01,36576,121533,A
1,"Gilman, Charlotte Perkins",The Yellow Wallpaper,1999-11-01,26363,6067,G
2,"Carroll, Lewis",Alice's Adventures in Wonderland,1994-03-01,18882,26389,C
3,"Shelley, Mary Wollstonecraft","Frankenstein; Or, The Modern Prometheus",2008-06-17,17128,74959,S
4,"Kafka, Franz",Metamorphosis,2005-08-16,15683,22022,K
...,...,...,...,...,...,...
76,"Maupassant, Guy de",Complete Original Short Stories of Guy De Maup...,2004-10-02,3437,490717,M
77,"Wood, Thomas",Practical Grammar and Composition,2007-09-11,3434,63677,W
78,"Dickens, Charles",David Copperfield,1996-12-01,3427,354823,D
79,"Rowlandson, Mary White",Narrative of the Captivity and Restoration of ...,1997-03-01,3389,20204,R


In [ ]:
# экспорт в базу данных
df.to_sql('books_json', engine, index=False, if_exists='replace', dtype={
    'name': sqlalchemy.VARCHAR(50),
    'title': sqlalchemy.VARCHAR(255),
    'downloads': sqlalchemy.Integer(),
    'words': sqlalchemy.Integer(),
    'publication': sqlalchemy.Date(),
    'Last_name_letter': sqlalchemy.CHAR()
})